# Sobrescribiendo properties con métodos plantilla

In [1]:
import iso6346

## Método plantilla

Como vimos en el notebook anterior, sobrescribir property getters y setters puede ser bastante lioso sintácticamente.

En este notebook vamos a emplear un patrón de diseño estándar llamado **método plantilla**.

## Property getter

Para hacer esto, vamos a meter los cálculos del volumen en un nuevo método llamado **_calcular_volumen**:

In [2]:
class ContenedorDeMercancias:
    
    # Definición de constantes de clase
    
    ALTURA = 8.5
    ANCHURA = 8.0
    
    # Definición de atributos de clase
    
    siguiente_codigo_de_serie = 1337
    
    @staticmethod
    def _crear_codigo_BIC(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6))
    
    @classmethod
    def _generar_codigo_de_serie(cls):
        
        resultado = cls.siguiente_codigo_de_serie 
        cls.siguiente_codigo_de_serie += 1
        return resultado
    
    # Crea un contenedor vacío
    @classmethod
    def crear_contenedor_vacio(cls, codigo_propietario, longitud, **kwargs):
        return cls(codigo_propietario, longitud, contenidos = [], **kwargs)
    
    # Crea un contenedor a partir de un codigo de propietario y unos items
    @classmethod
    def crear_contenedor_con_elementos(cls, codigo_propietario, longitud, contenidos, **kwargs):
        return cls(codigo_propietario, longitud, list(contenidos), **kwargs)
        
    
    def __init__(self, codigo_propietario, longitud, contenidos, **kwargs):
        
        # Definición de atributos de instancia
        
        self.codigo_propietario = codigo_propietario
        self.longitud = longitud
        self.contenidos = contenidos
        self.bic = ContenedorDeMercancias._crear_codigo_BIC(
            codigo_propietario=codigo_propietario,
            codigo_de_serie=ContenedorDeMercancias._generar_codigo_de_serie()
        )
        
    
    @property
    def volumen(self):
        return self._calcular_volumen()
    
    # Añadimos este nuevo método
    
    def _calcular_volumen(self):
        return ContenedorDeMercancias.ALTURA * ContenedorDeMercancias.ANCHURA * self.longitud

Ahora vamos a cambiar el property getter **volumen** a un método normal también llamado **_calcular_volumen**, sobrescribiendo el de la clase base que acabamos de definir:

In [3]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    MAX_CELSIUS = 4.0
    VOLUMEN_MAQUINA_REFRIGERADORA = 100
        
    def __init__(self, codigo_propietario, longitud, contenidos, *, celsius, **kwargs):
        super().__init__(codigo_propietario, longitud, contenidos, **kwargs)
        
        if celsius > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
        
        # Renombramos _celsius a celsius, autoencapsulándolo y accediendo a través de su property
        
        self.celsius = celsius
        
    # Creamos estos dos métodos para pasar de celsius a farenheit y viceversa
    
    @staticmethod
    def _celsius_to_farenheit(celsius):
        return celsius * 9/5 + 32
    
    @staticmethod
    def _farenheit_to_celsius(farenheit):
        return (farenheit -32) * 5/9 
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, valor):
        
        if valor > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
            
        self._celsius = valor
        
    # Creamos los getter y setter de farenheit
    
    @property
    def farenheit(self):
        return ContenedorDeMercanciasRefrigerado._celsius_to_farenheit(self.celsius)
    
    @farenheit.setter
    def farenheit(self, value):
        self.celsius = ContenedorDeMercanciasRefrigerado._farenheit_to_celsius(value)
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")
    
    """
    Sustituimos este property getter por _calcular_volumen
    
    @property
    def volumen(self):
        return super().volumen - ContenedorDeMercanciasRefrigerado.VOLUMEN_MAQUINA_REFRIGERADORA
    """
    
    def _calcular_volumen(self):
        return super()._calcular_volumen() - ContenedorDeMercanciasRefrigerado.VOLUMEN_MAQUINA_REFRIGERADORA

Gracias a este cambio, ahora cuando llamemos a volumen se ejecutará el property getter heredado de la clase base, que lo que hará será llamar al método **_calcular_volumen** de la propia clase gracias a **self**, evitando así tener que sobrescribir el property getter.

Por tener un ejemplo visual, ocurriría lo siguiente:

In [4]:
c = ContenedorDeMercancias.crear_contenedor_vacio("YML", longitud=8.0, celsius=0)

c.volumen

544.0

In [5]:
r = ContenedorDeMercanciasRefrigerado.crear_contenedor_vacio("YML", longitud=8.0, celsius=0)

r.volumen

444.0

**1.** Se ha llamado a ContenedorDeMercanciasRefrigerado.volumen, property getter heredado de la clase ContenedorDeMercancias
    
**2.** Este método ha devuelto self._calcular_volumen, que es ContenedorDeMercanciasRefrigerado._calcular_volumen

**3.** Se devuelve ContenedorDeMercancias._calcular_volumen - VOLUMEN_MAQUINA_REFRIGERADORA

### Resumen

Basicamente el método plantilla consiste en, en lugar de hacer los calculos en el property getter, realizarlos en un método adicional/plantilla. Dicho método será invocado por el property getter y sobrescrito para cada clase hija.

Como el property getter será heredado por cada una de las clases hija, todas podrán utilizarlo de forma homogénea, pero cada una de ellas ejecutará el método plantilla sobrescrito de forma específica para cada una de ellas. 

Todo esto sin necesidad de sobrescribir el property getter de ninguna forma.

## Property setter

Podemos utilizar la misma lógica para sobrescribir el property setter.

Esta vez lo haremos con el setter de celsius, que es el único que habíamos definido con una property.

Para ello, usaremos el método plantilla **_set_celsius**:

In [6]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    MAX_CELSIUS = 4.0
    VOLUMEN_MAQUINA_REFRIGERADORA = 100
        
    def __init__(self, codigo_propietario, longitud, contenidos, *, celsius, **kwargs):
        super().__init__(codigo_propietario, longitud, contenidos, **kwargs)
        
        if celsius > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
        
        # Renombramos _celsius a celsius, autoencapsulándolo y accediendo a través de su property
        
        self.celsius = celsius
        
    # Creamos estos dos métodos para pasar de celsius a farenheit y viceversa
    
    @staticmethod
    def _celsius_to_farenheit(celsius):
        return celsius * 9/5 + 32
    
    @staticmethod
    def _farenheit_to_celsius(farenheit):
        return (farenheit -32) * 5/9 
    
    @property
    def celsius(self):
        return self._celsius
    
    """
    Sustituimos este setter por el siguiente código
    
    @celsius.setter
    def celsius(self, valor):
        
        if valor > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
            
        self._celsius = valor
    """
    
    @celsius.setter
    def celsius(self, valor):
        return self._set_celsius(valor)
    
    def _set_celsius(self, valor):
        
        if valor > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
            
        self._celsius = valor
    
        
    # Creamos los getter y setter de farenheit
    
    @property
    def farenheit(self):
        return ContenedorDeMercanciasRefrigerado._celsius_to_farenheit(self.celsius)
    
    @farenheit.setter
    def farenheit(self, value):
        self.celsius = ContenedorDeMercanciasRefrigerado._farenheit_to_celsius(value)
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")
    
    def _calcular_volumen(self):
        return super()._calcular_volumen() - ContenedorDeMercanciasRefrigerado.VOLUMEN_MAQUINA_REFRIGERADORA

Ahora podemos eliminar la sintaxis liosa a la hora de sobrescribir el setter:

In [7]:
class ContenedorDeMercanciasRefrigeradoConCalefaccion(ContenedorDeMercanciasRefrigerado):
    
    MIN_CELSIUS = -20.0
    
    """
    En lugar del codigo comentado abajo, sobreescribimos el método plantilla _set_celsius
    
    @ContenedorDeMercanciasRefrigerado.celsius.setter
    def celsius(self, valor):
        
        if valor < ContenedorDeMercanciasRefrigeradoConCalefaccion.MIN_CELSIUS:
            raise ValueError("La temperatura es demasiado baja")
            
        ContenedorDeMercanciasRefrigerado.celsius.fset(self, valor)
    """
        
    def _set_celsius(self, valor):
        
        if valor < ContenedorDeMercanciasRefrigeradoConCalefaccion.MIN_CELSIUS:
            raise ValueError("La temperatura es demasiado baja")
            
        super()._set_celsius(valor)

A continuación podemos comprobar como funciona correctamente:

In [8]:
try:
    h = ContenedorDeMercanciasRefrigeradoConCalefaccion.crear_contenedor_vacio("YML", longitud=8.0, celsius=0)
    
except ValueError as error:
    print(error)
    
h.celsius

0

In [9]:
try:
    h = ContenedorDeMercanciasRefrigeradoConCalefaccion.crear_contenedor_vacio("YML", longitud=8.0, celsius=-30)
    
except ValueError as error:
    print(error)

La temperatura es demasiado baja


In [10]:
try:
    h = ContenedorDeMercanciasRefrigeradoConCalefaccion.crear_contenedor_vacio("YML", longitud=8.0, celsius=20)
    
except ValueError as error:
    print(error)

La temperatura es demasiado alta


Y aquí podemos ver como el property getter también funciona correctamente:

In [11]:
c.volumen

544.0

In [12]:
r.volumen

444.0

In [13]:
h.volumen

444.0